# 使用 pyresample 将弯曲网格 GOCI 子集重采样到 Landsat 规则网格

本 Notebook 以教学方式演示：
1. 读取已经按 Landsat footprint 精确裁剪后的 GOCI 子集（弯曲/曲线网格: curvilinear lat/lon）。
2. 读取 Landsat 多波段 TOA 辐亮度 GeoTIFF（规则栅格）。
3. 利用 `pyresample` 将 GOCI 五个波段插值/匹配到 Landsat 像元中心网格（两种算法：最近邻 / 高斯加权）。
4. 输出结果：多波段 GeoTIFF（主输出）与可选的逐波段 `.npy` 数组及元数据 JSON。
5. 做统计与可视化：差值统计、直方图、示例空间图。
6. 给出性能与内存优化提示，并封装 `run_all()` 供一键复现。

> 建议按顺序逐节运行与阅读。你可以先了解参数，再尝试修改重采样算法与半径。

## 1. 安装与导入库
如果你的环境还没有安装 `pyresample`, `rasterio`, `netCDF4`，运行下一单元进行安装。已安装可重复执行（pip 会跳过）。

In [1]:
# 安装可能缺失的库（静默）

import os, json, time, math
import numpy as np
import rasterio
from rasterio.transform import Affine
from rasterio.crs import CRS
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from pyproj import Transformer
from pyresample import geometry, kd_tree

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.dpi'] = 130
print('库导入完成。')

库导入完成。


## 2. 定义输入输出路径与参数
可以根据自己文件布局调整。重采样方法可以切换：`USE_GAUSSIAN=False` 表示最近邻 (KD-tree)；True 则使用高斯加权。半径参数需要根据 GOCI 原始分辨率（约 500 m）和 Landsat 30 m 调整。

In [4]:
# ---- 路径与参数可修改 ----
GOCI_NC = '../goci_subset_5bands.nc'  # 由裁剪脚本 20_goci_subset_rectangle.py 生成
LANDSAT_TIF = '../SR_Imagery/LC09_L1TP_116035_20250504_20250504_02_T1/LC09_L1TP_116035_20250504_20250504_02_T1_TOA_RAD_B1-2-3-4-5.tif'
OUT_TIF = '/goci_resampled_to_landsat.tif'
SAVE_NPY = True
NPY_DIR = 'resampled_npy'

# 波长映射 (GOCI 波段名与已裁剪 NC 中 geophysical_data 变量名保持一致)
GOCI_BANDS = {443:'L_TOA_443', 490:'L_TOA_490', 555:'L_TOA_555', 660:'L_TOA_660', 865:'L_TOA_865'}
LANDSAT_WAVELENGTHS = [443, 483, 561, 655, 865]  # Landsat TIF 中五个波段中心波长（近似）
PAIR_L2G = {0:443, 1:490, 2:555, 3:660, 4:865}  # Landsat 索引 -> 对应 GOCI 波长

# 重采样参数（可调）
NEAREST_RADIUS_M = 1200  # 最近邻搜索影响半径（米）
GAUSS_RADIUS_M   = 1500  # 高斯权重影响半径（米）
GAUSS_SIGMA      = 500   # 高斯 sigma（米）
FILL_VALUE = np.nan
USE_GAUSSIAN = False  # False=nearest, True=gaussian

os.makedirs(NPY_DIR, exist_ok=True)
print('参数已设置: 方法=', 'gaussian' if USE_GAUSSIAN else 'nearest')

参数已设置: 方法= nearest


## 3. 读取 GOCI 子集 NC（弯曲网格）
读取 latitude/longitude（curvilinear），以及 5 个波段数据；外部 FillValue 和掩膜 outside 区域替换为 `np.nan`，以便后续重采样忽略。

In [ ]:
def read_goci_subset(nc_path: str):
    if not os.path.exists(nc_path):
        raise FileNotFoundError(nc_path)
    with Dataset(nc_path, 'r') as ds:
        nav = ds['navigation_data']
        geo = ds['geophysical_data']
        lat = np.array(nav['latitude'][:], dtype=np.float32)
        lon = np.array(nav['longitude'][:], dtype=np.float32)
        inside = None
        bands = {}
        for wl, vname in GOCI_BANDS.items():
            var = geo[vname]
            data = var[:]
            if np.ma.isMaskedArray(data):
                arr = data.filled(np.nan).astype(np.float32)
            else:
                arr = np.array(data, dtype=np.float32)
            fv = None
            if '_FillValue' in var.ncattrs():
                try:
                    fv = float(var.getncattr('_FillValue'))
                except Exception:
                    fv = None
            if fv is not None:
                arr = np.where(arr == fv, np.nan, arr)
            if inside is not None:
                arr = np.where(inside, arr, np.nan)
            bands[wl] = arr
    print('GOCI 子集读取完成: shape=', lat.shape, '波段有效像元统计:')
    for wl in sorted(bands):
        valid = np.isfinite(bands[wl]).sum()
        print(f'  wl {wl}nm: valid={valid}')
    return {'lat':lat, 'lon':lon, 'bands':bands, 'inside':inside}

goci_data = read_goci_subset(GOCI_NC)

GOCI 子集读取完成: shape= (669, 901) 波段有效像元统计:
  wl 443nm: valid=412528
  wl 490nm: valid=412528
  wl 555nm: valid=412528
  wl 660nm: valid=412528
  wl 865nm: valid=412528


## 4. 读取 Landsat 多波段 TIF 并构建目标网格
读取 5 波段数组及其仿射变换，计算像元中心经纬度（WGS84）。

In [ ]:
def read_landsat_tif(tif_path: str):
    if not os.path.exists(tif_path):
        raise FileNotFoundError(tif_path)
    with rasterio.open(tif_path) as ds:
        stack = ds.read().astype(np.float32)  # (5,H,W)
        if ds.nodata is not None:
            stack = np.where(stack == ds.nodata, np.nan, stack)
        H, W = ds.height, ds.width
        T = ds.transform
        crs = ds.crs
        rows = np.arange(H); cols = np.arange(W)
        cgrid, rgrid = np.meshgrid(cols, rows)
        # 像元中心投影坐标
        x = T.c + T.a*(cgrid + 0.5) + T.b*(rgrid + 0.5)
        y = T.f + T.d*(cgrid + 0.5) + T.e*(rgrid + 0.5)
        transformer = Transformer.from_crs(crs, 'EPSG:4326', always_xy=True)
        lon, lat = transformer.transform(x, y)
    print('Landsat 读取完成: shape=', stack.shape,
          'lon范围=(', np.nanmin(lon), ',', np.nanmax(lon), '), lat范围=(', np.nanmin(lat), ',', np.nanmax(lat), ')')
    return {'data': stack, 'lon': lon, 'lat': lat, 'transform': T, 'crs': crs, 'dtype': stack.dtype}

landsat_data = read_landsat_tif(LANDSAT_TIF)

## 5. 构建 pyresample 几何对象 (SwathDefinition)
源：GOCI 弯曲网格 (lat, lon)；目标：Landsat 像元中心 (lat, lon)。本例直接用两个 SwathDefinition；如果需要投影规则网格，可改用 AreaDefinition。

In [ ]:
g_lon = goci_data['lon']; g_lat = goci_data['lat']
L_lon = landsat_data['lon']; L_lat = landsat_data['lat']
source_swath = geometry.SwathDefinition(lons=g_lon, lats=g_lat)
target_swath = geometry.SwathDefinition(lons=L_lon, lats=L_lat)
print('几何对象已构建: source=', g_lon.shape, 'target=', L_lon.shape)

## 6. 选择与配置重采样算法 (nearest / gaussian)
`resample_nearest`：快速、保留原值；`resample_gauss`：平滑、可减噪。使用时需合理设置 `radius_of_influence` 与 `sigmas`。

In [ ]:
def choose_resampler(use_gaussian: bool):
    if use_gaussian:
        def _resample(src_swath, dst_swath, data):
            return kd_tree.resample_gauss(
                src_swath, data, dst_swath,
                radius_of_influence=GAUSS_RADIUS_M,
                sigmas=GAUSS_SIGMA,
                fill_value=FILL_VALUE,
                neighbours=32
            )
        method = 'gaussian'
    else:
        def _resample(src_swath, dst_swath, data):
            return kd_tree.resample_nearest(
                src_swath, data, dst_swath,
                radius_of_influence=NEAREST_RADIUS_M,
                fill_value=FILL_VALUE,
                neighbours=1
            )
        method = 'nearest'
    return _resample, method

resample_func, RESAMPLE_METHOD = choose_resampler(USE_GAUSSIAN)
print('使用方法:', RESAMPLE_METHOD)

## 7. 执行重采样：批处理 5 个 GOCI 波段
循环处理每个波段，记录时间与统计。

In [ ]:
resampled_arrays = []  # 按升序波长存放
resampled_wls = sorted(GOCI_BANDS.keys())
start_all = time.time()
for wl in resampled_wls:
    arr = goci_data['bands'][wl]
    t0 = time.time()
    out = resample_func(source_swath, target_swath, arr)
    dt = time.time() - t0
    valid = np.isfinite(out).sum()
    print(f'Resample wl {wl}nm done: valid={valid}, time={dt:.2f}s')
    resampled_arrays.append(out.astype(np.float32))

print('全部重采样完成，总耗时: %.2fs' % (time.time() - start_all))

## 8. 保存结果为多波段 GeoTIFF
使用 Landsat 原 TIF 的空间参考与变换，写入浮点多波段。

In [ ]:
def write_resampled_tif(out_path: str, arrays: list, landsat_meta: dict, wavelengths):
    if len(arrays) == 0:
        raise ValueError('arrays 为空')
    h, w = arrays[0].shape
    profile = {
        'driver': 'GTiff',
        'height': h,
        'width': w,
        'count': len(arrays),
        'dtype': 'float32',
        'crs': landsat_meta['crs'],
        'transform': landsat_meta['transform'],
        'compress': 'deflate',
        'nodata': np.nan
    }
    with rasterio.open(out_path, 'w', **profile) as dst:
        for i, arr in enumerate(arrays, start=1):
            dst.write(arr, i)
        dst.update_tags(1, wavelengths=','.join(str(x) for x in wavelengths))
        dst.update_tags(source='GOCI->Landsat grid via pyresample', method=RESAMPLE_METHOD)
    print('写入 GeoTIFF 完成:', out_path)

write_resampled_tif(OUT_TIF, resampled_arrays, landsat_data, resampled_wls)

## 9. 可选：保存结果为 .npy
逐波段保存，并写 meta.json。

In [ ]:
if SAVE_NPY:
    meta = {
        'wavelengths': resampled_wls,
        'method': RESAMPLE_METHOD,
        'shape': list(resampled_arrays[0].shape),
        'timestamp': time.strftime('%Y-%m-%dT%H:%M:%S'),
        'radius': GAUSS_RADIUS_M if USE_GAUSSIAN else NEAREST_RADIUS_M,
        'sigma': GAUSS_SIGMA if USE_GAUSSIAN else None
    }
    for wl, arr in zip(resampled_wls, resampled_arrays):
        np.save(os.path.join(NPY_DIR, f'goci_resample_{wl}nm.npy'), arr)
    with open(os.path.join(NPY_DIR, 'meta.json'), 'w') as f:
        json.dump(meta, f, indent=2)
    print('NPY 与 meta.json 已保存到', NPY_DIR)
else:
    print('SAVE_NPY=False，跳过 NPY 保存')

## 10. 波段配对统计（重采样 GOCI vs Landsat）
计算差值、RMSE、相关系数。

In [ ]:
def pairwise_stats(resampled_list, landsat_stack):
    print('Wavelength  |   N   | mean(G) mean(L) |  diff mean  diff std  RMSE  |  corr')
    print('-'*85)
    for li, wl in enumerate(resampled_wls):
        L_index = None
        for k,v in PAIR_L2G.items():
            if v == wl:
                L_index = k
                break
        if L_index is None:
            continue
        G = resampled_list[li]
        L = landsat_stack[L_index]
        mask = np.isfinite(G) & np.isfinite(L)
        if mask.sum() == 0:
            print(f'{wl:>9}nm |   0   | NA')
            continue
        g = G[mask]; l = L[mask]
        diff = g - l
        rmse = np.sqrt(np.mean(diff**2))
        corr = np.corrcoef(g, l)[0,1]
        print(f'{wl:>9}nm | {mask.sum():5d} | {g.mean():7.4f} {l.mean():7.4f} | {diff.mean():9.4f} {diff.std():8.4f} {rmse:6.4f} | {corr:5.3f}')

pairwise_stats(resampled_arrays, landsat_data['data'])

## 11. 快速可视化：重采样 GOCI vs Landsat
示例 555 nm（可修改 wl）。

In [ ]:
def show_example(wl=555):
    if wl not in resampled_wls:
        print('波长不存在于重采样列表'); return
    # 找到 Landsat 对应索引
    L_index = None
    for k,v in PAIR_L2G.items():
        if v == wl:
            L_index = k
            break
    if L_index is None:
        print('未找到 Landsat 对应波段'); return
    G = resampled_arrays[resampled_wls.index(wl)]
    L = landsat_data['data'][L_index]
    lon = L_lon; lat = L_lat
    both = np.isfinite(G) & np.isfinite(L)
    if both.sum() == 0:
        print('无共同有效像元'); return
    vv = np.concatenate([G[both], L[both]])
    vmin, vmax = np.percentile(vv, [2, 98])
    cmap = plt.get_cmap('viridis').copy(); cmap.set_bad(alpha=0.)
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.title(f'GOCI resampled {wl}nm ({RESAMPLE_METHOD})')
    plt.pcolormesh(lon, lat, G, shading='auto', vmin=vmin, vmax=vmax, cmap=cmap)
    plt.colorbar(label='Radiance')
    plt.subplot(1,2,2)
    plt.title(f'Landsat ~{wl}nm')
    plt.pcolormesh(lon, lat, L, shading='auto', vmin=vmin, vmax=vmax, cmap=cmap)
    plt.colorbar(label='Radiance')
    plt.tight_layout()
    plt.show()

show_example(555)

## 12. 直方图对比（重采样 GOCI vs Landsat）
与先前脚本风格一致：共用 1–99% 分位范围，`bins=60`。

In [ ]:
hist_dir = 'figs_hist_resample'
os.makedirs(hist_dir, exist_ok=True)

def robust_range(arrs):
    valid = [a[np.isfinite(a)] for a in arrs if a is not None]
    valid = [v for v in valid if v.size]
    if not valid: return (0,1)
    vv = np.concatenate(valid)
    return np.percentile(vv, [1,99])

for li, wl in enumerate(resampled_wls):
    # 找 Landsat index
    L_index = None
    for k,v in PAIR_L2G.items():
        if v == wl:
            L_index = k; break
    if L_index is None: continue
    G = resampled_arrays[li]
    L = landsat_data['data'][L_index]
    maskG = np.isfinite(G); maskL = np.isfinite(L)
    if maskG.sum()==0 or maskL.sum()==0:
        print('跳过 wl', wl, '有效像元不足'); continue
    vmin, vmax = robust_range([G[maskG], L[maskL]])
    plt.figure(figsize=(6.4,4.2))
    plt.hist(L[maskL], bins=60, range=(vmin,vmax), alpha=0.6, label='Landsat', density=True)
    plt.hist(G[maskG], bins=60, range=(vmin,vmax), alpha=0.6, label='GOCI resampled', density=True)
    plt.title(f'Histogram ~{wl}nm ({RESAMPLE_METHOD})')
    plt.xlabel('Radiance')
    plt.ylabel('Density')
    plt.legend()
    plt.tight_layout()
    outp = os.path.join(hist_dir, f'hist_resample_{wl}nm.png')
    plt.savefig(outp, dpi=130)
    plt.close()
    print('保存:', outp)

## 13. 性能与内存优化建议
1. 半径不要过大：500 m 分辨率源 -> 1~2 km 半径通常足够；过大显著增加 KD 树查询耗时。
2. 逐波段即写：若内存紧张，可重采样一个写一个，不缓存 `resampled_arrays`。
3. 使用 `resample_nearest` 做初步分析，确认流程后再尝试 `gauss`。
4. 预先把源 outside 区域设为 NaN（已做），减少参与计算的点数。
5. 可尝试分块重采样（需自实现窗口切片 + 组合），或考虑 dask + pyresample（高级）。
6. 若后续仅做统计回归，可降采样目标网格 (e.g., 每 2~3 像元取一) 加速。

## 14. 封装一键运行函数
方便你修改参数后直接重新跑完整流程。

In [ ]:
def run_all():
    # 重新读取（假设可能修改参数后）
    g = read_goci_subset(GOCI_NC)
    Ld = read_landsat_tif(LANDSAT_TIF)
    src = geometry.SwathDefinition(lons=g['lon'], lats=g['lat'])
    dst = geometry.SwathDefinition(lons=Ld['lon'], lats=Ld['lat'])
    func, method = choose_resampler(USE_GAUSSIAN)
    arrays = []
    for wl in sorted(GOCI_BANDS.keys()):
        arr = g['bands'][wl]
        arrays.append(func(src, dst, arr).astype(np.float32))
    write_resampled_tif(OUT_TIF, arrays, Ld, sorted(GOCI_BANDS.keys()))
    if SAVE_NPY:
        meta = {'wavelengths': sorted(GOCI_BANDS.keys()), 'method': method}
        for wl, arr in zip(sorted(GOCI_BANDS.keys()), arrays):
            np.save(os.path.join(NPY_DIR, f'goci_resample_{wl}nm.npy'), arr)
        with open(os.path.join(NPY_DIR, 'meta.json'), 'w') as f:
            json.dump(meta, f, indent=2)
    pairwise_stats(arrays, Ld['data'])
    print('run_all 完成')

# 直接调用快速运行（如不需要可注释）
# run_all()